In [9]:
# Importing libraries
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from lxml.html import fromstring
import urllib.request
import requests
import time
import os

In [10]:
obj = input('Enter object: ')                                    # Must have + between multiple words
num_vid = input('Enter number of videos to scrape: ')
url_vid = "https://www.shutterstock.com/video/search/" + obj

Enter object: Guitar
Enter number of videos to scrape: 5


In [27]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'}

In [28]:
# Load and scrape comments
def vid_scrape(url):
    driver = webdriver.Chrome()
    driver.maximize_window()
    for i in range(1, int(num_vid) + 1):
        driver.get(url)
        assert "Shutterstock" in driver.title  # Only works for valid Shutterstock pages
        oldHeight = driver.execute_script("return document.documentElement.scrollHeight")
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="content"]/div[2]/div/div[2]/div/main/div/div[3]/div[' + str(i) + ']/a')))
        except:
            driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
            newHeight = driver.execute_script("return document.documentElement.scrollHeight")
            if newHeight == oldHeight:
                driver.close()
                return
            oldHeight = newHeight
        while True:
            vid_retrieve = driver.find_elements_by_xpath('//*[@id="content"]/div[2]/div/div[2]/div/main/div/div[3]/div[' + str(i) + ']/a')
            print(vid_retrieve)
            if len(vid_retrieve) != 0:
                break
        vid_retrieve[0].click()
        time.sleep(10)  # Buffer time
        start_time = time.time()
        while True:
            WebDriverWait(driver, 30).until(EC.visibility_of_element_located((By.XPATH, "//video[@data-automation='VideoPlayer_video_video']")))
            vid_indURL = driver.current_url
            response = requests.get(vid_indURL, headers = headers)
            tree = fromstring(response.content)
            vid_link = tree.xpath('//*[@id="content"]/div[2]/div/div[2]/div/main/div/div/div/div[2]/div[1]/div/div[1]/div/video/source[2]/@src')  # List
            print(vid_link)
            elapsed_time = time.time() - start_time
            print(elapsed_time)
            if elapsed_time > 20:
                break
            if len(vid_link) != 0:
                break
        if len(vid_link) > 0:
            vfile_path = os.path.join(r".\Videos", 'vid_ ' + str(i) + '.mp4')
            urllib.request.urlretrieve(vid_link[0], vfile_path)  # Save each video to Videos directory

In [29]:
vid_scrape(url_vid)

[<selenium.webdriver.remote.webelement.WebElement (session="8a39ccb5ee8ce17bd2ee5a1ea3125051", element="0.4265932818664335-1")>]
['https://ak4.picdn.net/shutterstock/videos/1018015894/preview/stock-footage-sitting-at-the-dinner-table-handsome-young-man-plays-the-guitar-for-a-friends-family-and-friends.mp4']
0.381716251373291
[<selenium.webdriver.remote.webelement.WebElement (session="8a39ccb5ee8ce17bd2ee5a1ea3125051", element="0.5771839748309395-1")>]
['https://ak3.picdn.net/shutterstock/videos/1014841363/preview/stock-footage-funny-gorillas-and-monkeys-play-on-guitar-and-drums-rock-party-on-sunny-seaside-realistic-k.mp4']
0.44266557693481445
[<selenium.webdriver.remote.webelement.WebElement (session="8a39ccb5ee8ce17bd2ee5a1ea3125051", element="0.43094963225110017-1")>]
['https://ak6.picdn.net/shutterstock/videos/1018298596/preview/stock-footage-big-family-garden-party-celebration-gathered-together-at-the-table-relatives-and-friends-young.mp4']
0.3650805950164795
[<selenium.webdriver.r